In [2]:
from flask import Flask, jsonify
from db import db
import pandas as pd
import constants
import pickle

In [ ]:
for x in db.nba_teams.find():
    db.nba_teams.insert_one({
        "_id": x["_id"],
        "abbr": x["abbr"],
        "name": x["name"],
        "short": x["short"],
        "city": x["city"],
        "roster": x["roster"],
    })
    db.nba_teams.delete_one({"_id": x["_id"]})

In [ ]:
# Update leaderboards db with stat names
for x in db.nba_leaderboards.find():
    db.nba_leaderboards.update_one({"_id": x["_id"]}, {"$set": {"name": constants.leaderboard_full_names[f'leaders_{x["_id"]}']}})

# Insert roster document for each team
for x in db.rosters.find():
    for y in x:
        if y == "_id":
            continue
        db.rosters.insert_one({"_id": y, "roster": x[y]})

# Insert document for each game in schedule db
for x in db.nba_schedule.find():
    for y in x:
        if y == "_id":
            continue
        db.nba_schedule.insert_one({"_id": y, "schedule": x[y]})

# Insert document for each player in player_data db
for x in db.player_data.find():
    for y in x:
        if y == "_id":
            continue
        db.player_data.insert_one({"_id": y, "data": x[y]})


# Update rosters collections in mongodb
with open(f"backend//player_data//player_data.txt", "r") as f:
    player_data_dict = json.load(f)
db.player_data.insert_one(player_data_dict)
with open(f"backend//rosters//team_rosters.txt", "r") as f:
    team_roster_dict = json.load(f)
db.rosters.insert_one(team_roster_dict)


# Update leaderboard collections in mongodb
for stat in constants.leaderboard_names:
    leaderboard = bbref.get_leaderboards(stat, 2022).to_dict(orient='list')
    leaderboard.update({"_id": f"{stat.replace('leaders_', '')}"})
    with open(f'leaderboards\{stat}.pkl', 'wb') as f:
        pickle.dump(leaderboard, f)
    print(f'{stat} done')

for file in os.listdir('leaderboards'):
    with open(f'leaderboards\{file}', 'rb') as f:
        leaderboard = pickle.load(f)
    db.nba_leaderboards.insert_one(leaderboard)


# Update schedule collection in mongodb
db.nba_schedule.delete_one({"_id": "schedule_2023"})
schedule = bbref.get_schedule(2023).to_dict(orient='index')
schedule.update({"_id": "schedule_2023"})
db.nba_schedule.insert_one(schedule)

In [ ]:
# Move leaderboards 2 to leaderboards db
for x in db.nba_leaderboards.find():
    db.nba_leaderboards.delete_one({"_id": x["_id"]})
for x in db.nba_leaderboards2.find():
    db.nba_leaderboards.insert_one({
        "_id": x["_id"],
        "player_id": x["player_id"],
        "value": x["value"],
        "per_mode": x["per_mode"],
    })

In [ ]:
# Add team icons to db
for key in constants.team_icons:
    db.nba_teams.update_one({"abbr": key}, {"$set": {"icon": constants.team_icons[key]}})

In [6]:
# Copy teams collection to nba_teams collection
for x in db.nba_schedule.find():
    db.nba_schedule.insert_one(x)